In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpz7ca7q6s/pubring.gpg' created
gpg: /tmp/tmpz7ca7q6s/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


## gensim word2vec preparation

In [3]:
!pip install -U gensim
#we use this for the word2vec representations of each word
#remove if not using google colab. Instead, install from console

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.4.0)
Requirement not upgraded as not directly required: six>=1.5.0 in /usr/local/lib/python3.6/dist-packages (from gensim) (1.11.0)
Requirement not upgraded as not directly required: scipy>=0.18.1 in /usr/local/lib/python3.6/dist-packages (from gensim) (0.19.1)
Requirement not upgraded as not directly required: smart-open>=1.2.1 in /usr/local/lib/python3.6/dist-packages (from gensim) (1.6.0)
Requirement not upgraded as not directly required: numpy>=1.11.3 in /usr/local/lib/python3.6/dist-packages (from gensim) (1.14.5)
Requirement not upgraded as not directly required: boto3 in /usr/local/lib/python3.6/dist-packages (from smart-open>=1.2.1->gensim) (1.7.50)
Requirement not upgraded as not directly required: bz2file in /usr/local/lib/python3.6/dist-packages (from smart-open>=1.2.1->gensim) (0.98)
Requirement not upgraded as not directly required: boto>=2.32 in /usr/local/lib/python3.6/dist-packages (from

use gensim to get the word2vec representations of each word  
reference here:  
https://www.quora.com/How-do-you-extract-vectors-from-word2vec-given-a-word

In [0]:
from gensim.models import KeyedVectors#, Word2Vec

#currently using gensim's default word2vec pretrained model, trained on googlenews data. 
#todo: train on wikiqa if the results are crap. 

w2vmodel = KeyedVectors.load_word2vec_format('drive/Colab Notebooks/datasets/word2vec/GoogleNews-vectors-negative300-SLIM.bin.gz', binary=True)

word2vec first testing

In [5]:
# print(w2vmodel['computer'])
# print(w2vmodel.similarity('woman','man'))
wvdict = w2vmodel.wv.vocab
#this is the list of all the words in the vocabulary

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


## Preprocessing input strings

In [0]:
import numpy as np

`a.toks` contains the questions (repeat question lines correspond to the number of candidate answer sentences)  
`b.toks` contains the answers. Each line is the answer to the question on the same line of `a.toks`  
`id` contains the id of the question at that line in `a.toks`  
`sim` contains the labels: for each block of repeated-questions: there is one line containing a `1` corresponding to the line in `b.toks` containing the correct answer to the same line in `a.toks`

https://github.com/castorini/data/tree/master/WikiQA

In [0]:
def read_data(path, readtype):
  with open(path) as f:
    lines = f.readlines()
    
  if readtype == "questions" or readtype == "answers":
    return lines
  elif readtype == "labels":
    return [int(line) for line in lines]

Load the data

In [0]:
bigpath = "drive/Colab Notebooks/datasets/QuestionAnswering/wikiQA"
questions = read_data(bigpath+"/train/a.toks", "questions")
answers = read_data(bigpath+"/train/b.toks", "answers")
labels = read_data(bigpath+"/train/sim.txt", "labels")
#question_ids = read_data(bigpath+"/train/id.txt", "labels")
#questions

#### don't use this 

Next, generate an `embedding_matrix`.

Each sentence is represented by the average of its word vectors.  
Each row of this matrix will be a **600** length dual-word-vector, because one half is the questions, the other half is the answer.


In [0]:
# ###THIS IS not right
# question_wvs = []
# answer_wvs = []
# processed_qids = []

# for item in range(len(questions)):
#   if !(question_ids[item] in processed_qids):
#     question_wvs.append( list(np.mean(np.vstack(tuple(w2vmodel[word] for word in questions[item])), axis=1) ) )
#     #stack the word vectors (of all the words in a sentence) on top of each other, and run a columnwise np.mean to find the sentence-level word vector
#     processed_qids.append(question_ids[item])
#   else:
#     question_wvs.append( question_wvs[item-1] ) #if the word vector of a question was already processed (because there are duplicate questions in the file), get the vector of the previous calculation which should be the same
  
#   #now let's work on the answers
#   answer_wvs.append( list(np.mean(np.vstack(tuple(w2vmodel[word] for word in answers[item])), axis=1)) )
  
  
# #turn the lists of np arrays into... np arrays (matrices to be exact, each row is a BoW-averaged word vector)
# question_wvs_arr = np.array(question_wvs)
# answer_wvs_arr = np.array(answer_wvs)

# #concatenate side by side the question matrix and the answer matrix to obtain the embedding_matrix
# embedding_matrix = np.concatenate((question_wvs_arr,answer_wvs_arr), axis = 1)

# #check size, should have 600 columns
# embedding_matrix.shape





#### preparing word-level sequence embedding

This is to prepare for the Embedding layer

1. concatenate questions and answers into a list of `[question | answer]` pairs. This is a list of sequences.  
2. find max sequence length from this list  
3. the vocab size of the google news word2vec is 3 million words, the number of features is 300

The embedding_matrix:
1. let each word in `w2vmodel.wv.vocab.keys()` have an index
2. concatenate axis=1 all the word vectors in the vocabulary into embedding_matrix
3. then turn each string sequence into an array of numbers (each number represents the index of that word in the vocab)
4. pass the string sequences into the .fit as training input. The shape of this input should be (n_seqs, max_seq_length, n_features)
5. the embedding matrix will be the weights in the Embedding layer

In [0]:
vocab_size = 299567 #this is the SLIMmed googlenews w2v model's vocab count

In [10]:
#make embedding matrix
embedding_matrix = np.zeros((vocab_size, 300))
for i in range(len(wvdict)):
    embedding_vector = w2vmodel.wv[w2vmodel.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


#### define some functions to preprocess data (both train and test)

In [0]:
def SEQ_max_length(sequences):
  return max( [len(qa) for qa in sequences] )

In [0]:
#used for versions 0
def SEQ_make_qa_sequences_list(questions, answers):
  #remove question marks if there is one
  questions = [qu.replace(' ?', '') for qu in questions]
  #concatenate question and answer
  sequences = [qu+' '+ans for qu, ans in zip(questions, answers)]
  
  return sequences

# def STR_purge(s):
#   s = ''.join(i for i in s if not i.isdigit())
#   s = ''.join(i for i in s if not (i in ['of']))
#   return s

# questions = [qu.replace(' ?', '') for qu in questions]
# questions = [STR_purge(qu) for qu in questions]

#for version 0
#sequences = SEQ_make_qa_sequences_list(questions, answers)
#max_seq_length = SEQ_max_length(sequences)


In [13]:
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
def SEQ_token_and_pad(sequences, max_seq_length):
  
#   #version 0a
#   t = Tokenizer()
#   t.fit_on_texts(sequences)

#   vocab_size = len(t.word_index)+1

#   encoded_seqs = t.texts_to_sequences(sequences)
#   print(encoded_seqs)

#   #version 0b and up
  encoded_seqs = []
  for qa in sequences:
    encoded_qa = []
    tokenized_qa = qa.split()
    for word in tokenized_qa:
      index_to_append = 0
      
      if word in wvdict.keys():
        index_to_append = w2vmodel.vocab[word].index
      encoded_qa.append(index_to_append)
    
    encoded_seqs.append(encoded_qa)
    
  print(encoded_seqs)
  
  padded_seqs = pad_sequences(encoded_seqs, maxlen = max_seq_length, padding="post")
  return padded_seqs

#version 0
#padded_seqs = SEQ_token_and_pad(sequences, max_seq_length)


Performing the necessary preprocessing on our data using our functions

In [15]:

max_question_length = SEQ_max_length(questions)
max_answer_length = SEQ_max_length(answers)
max_hidden_length = max(max_question_length, max_answer_length)

#version 1: getting separate questions and answers vectors (but still encoded in index form, not in pure word vector form)
padded_questions = SEQ_token_and_pad(questions, max_hidden_length)
padded_answers = SEQ_token_and_pad(answers, max_hidden_length)


[[132, 17, 27136, 22292, 2874, 0], [132, 17, 27136, 22292, 2874, 0], [132, 17, 27136, 22292, 2874, 0], [132, 17, 27136, 22292, 2874, 0], [132, 17, 27136, 22292, 2874, 0], [132, 141, 3, 0, 37654, 0, 369], [132, 141, 3, 0, 37654, 0, 369], [132, 141, 3, 0, 37654, 0, 369], [132, 141, 3, 0, 37654, 0, 369], [132, 141, 3, 0, 37654, 0, 369], [132, 141, 3, 0, 37654, 0, 369], [132, 141, 3, 0, 37654, 0, 369], [132, 141, 3, 0, 37654, 0, 369], [132, 141, 3, 0, 37654, 0, 369], [132, 141, 3, 0, 37654, 0, 369], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132

[[0, 4602, 16068, 27136, 12819, 4, 0, 255051, 27136, 0], [9, 2205, 19554, 3, 1254, 0, 6638, 137], [2205, 20419, 0, 9, 0, 27136, 12819], [0, 27136, 12819, 3, 0, 12819, 2874, 380, 9, 2205, 0, 0, 27136, 0], [27136, 22292, 17, 608, 250, 2205, 22292, 0, 30, 25, 460, 3, 3455, 219, 0, 4480, 24223, 22292, 2, 3052, 0, 2205, 0], [25, 37654, 22, 0, 1479, 0, 41, 0, 38212, 0], [8748, 36943], [0, 9, 154, 0, 1244, 0, 61962, 0, 0, 37654, 3, 9, 548, 1426, 0, 25515, 219, 1, 3658, 15, 0, 4541, 0], [0, 9, 46492, 0, 76723, 0, 100, 19708, 496, 0, 0, 37654, 3, 0, 1426, 0, 527, 25515, 1536, 219, 1, 1654, 0], [0, 496, 111, 0, 37654, 3, 0, 1654, 25515, 0, 9, 10026, 3851, 0, 9, 154, 37654, 3, 768, 9, 12198, 25515, 26, 9, 11770, 25515, 0], [0, 195530, 0, 36821, 9, 1479, 0, 40, 37654, 0, 3, 6936, 219, 12, 0, 1488, 0, 2032, 0, 5038, 0], [13, 3, 27365, 12, 7451, 0, 145385, 0, 131274, 0, 76476, 0, 0, 0, 26, 0, 0], [9, 1479, 0, 4342, 36546, 3, 11, 7914, 16, 379, 0, 3, 1214, 0, 5007, 13528, 0], [0, 9, 73282, 40, 37654,

In [0]:
#version 2: generate a (n_qapairs, max_question_length OR max_answer_length, 300) for questions and answers. Feed these two tensors into the Input layer of each LSTM. 
#not yet

## Model building and training

### Guesswork and experimentation



---


*If i'm doing the implementation in the 1506.06490 paper*  
**Todo**: 

- find length of the longest question and longest answer
- use these maxes for padding to shorter sentences
- in one big model, build 2 separate convnet submodels, one for questions, one for answers
- concatenate the outputs for the questions and answers convnets, and we obtain the **joint representation** `p` of each question-answer pair.

Each of these learned joint pairs is a single **timestep** in the LSTM part. We learn the sequence and semantic relationships between each answer-candidate sentence in the context paragraph, not between the words themselves. (the word-level patterns are learned in the Conv stage already)

for the LSTM stage, padding instead involves adding dummy qa pairs for each timestep without an answer. The max number of answer candidates in the dataset (and possible qa pairs, per question) is 30, so any Qs with fewer than 30 answer sentences will be padded with dummy pairs.


---



*my own simple-ass idea*  
**Todo:**

version 0a: `shit lstm model copped from some blog`  
https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

version 0b: `with learning rate, and fixed word tokenizations`

version 1a: `separate LSTM for question and answer, then Concatenate into a hidden layer which then is fed into classifier/Dense/whatever`

In [0]:
from keras.models import Sequential
from keras.layers import LSTM, Embedding
from keras.optimizers import Adam

#build the model - version0a: shit lstm model copied online
model = Sequential()
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_seq_length, trainable=False)
model.add(e)
model.add(LSTM(100))
#model.add(Dropout(0.2))
model.add(Dense(1,activation="sigmoid"))

In [36]:
#version 1a: suggestion from Giang
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation, Embedding
from keras.layers import Merge
from keras.optimizers import Adam

equestions = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_hidden_length, trainable=False)
eanswers = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_hidden_length, trainable=False)
# doc question
# input_1 = Input(...)
# ...
# output_1 = ...


# # # doc answer
# # input_2 = Input(...)
# # ...
# # output_2 = ...
# input_2 = Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_answer_length, trainable=False)
# input_reshape_2 = Reshape((None, max_question_length, 300))(input_2)
# output_2 = LSTM(100)(input_reshape_2)

# hidden = Concatenate()([output_1, output_2])
# classifier = Dense(1, activation="sigmoid")(hidden)


questions_model = Sequential()
questions_model.add(equestions)
questions_model.add(LSTM(64, return_sequences=False, activation='softmax'))
# questions_model.add(Activation('softmax'))
questions_model.add(Dropout(0.5))

answers_model = Sequential()
answers_model.add(eanswers)
answers_model.add(LSTM(64, return_sequences=False, activation='softmax'))
# answers_model.add(Activation('softmax'))
answers_model.add(Dropout(0.5))

merged = Merge([questions_model, answers_model], mode = "concat", concat_axis = -1)

final_model = Sequential()
final_model.add(merged)
final_model.add(Dense(1, activation="softmax"))

final_model.compile(loss = 'binary_crossentropy', optimizer = Adam(), metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


### Attempting the 1506 06490 paper

In [0]:
from keras.layers import Dense, LSTM, Conv1D, Embedding, MaxPooling1D, Activation, Merge
from keras.models import Sequential
from keras.optimizers import Adadelta

In [39]:
#embedding is Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_hidden_length, trainable = False)

questions_model = Sequential()
questions_model.add(Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_hidden_length, trainable = False))
questions_model.add(Conv1D(100, 5, strides=1))
questions_model.add(MaxPooling1D(pool_size=2))
questions_model.add(Activation('relu'))

questions_model.add(Conv1D(100, 3, strides=1))
questions_model.add(MaxPooling1D(pool_size=2))
questions_model.add(Activation('relu'))

questions_model.add(Conv1D(100, 3, strides=1))
questions_model.add(MaxPooling1D(pool_size=1))
questions_model.add(Activation('relu'))

answers_model = Sequential()
answers_model.add(Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_hidden_length, trainable = False))
answers_model.add(Conv1D(100, 5, strides=1))
answers_model.add(MaxPooling1D(pool_size=2))
answers_model.add(Activation('relu'))

answers_model.add(Conv1D(100, 3, strides=1))
answers_model.add(MaxPooling1D(pool_size=2))
answers_model.add(Activation('relu'))

answers_model.add(Conv1D(100, 3, strides=1))
answers_model.add(MaxPooling1D(pool_size=1))
answers_model.add(Activation('relu'))

merge = Merge([questions_model, answers_model])

final_model = Sequential()
final_model.add(merge)
final_model.add(LSTM(160))
final_model.add(Dense(1, activation="sigmoid"))

final_model.compile(loss = 'binary_crossentropy', optimizer = Adadelta(lr=1), metrics=['accuracy'])
print(final_model.summary())


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_6 (Merge)              (None, 74, 100)           0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 160)               167040    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 161       
Total params: 180,328,001
Trainable params: 587,801
Non-trainable params: 179,740,200
_________________________________________________________________
None


## Fit the chosen model

Severely punish "no" answers because that's just lazy. The network should at least try to guess "yes" or "1" on at least some questions, and not settle for a 88% accuracy by answering no to everything.

add class_weights and try:

In [0]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced', np.unique(labels), labels)
#class_weights = np.array([0.5, 24.])

In [40]:
#final_model.fit([padded_questions, padded_answers], labels, epochs=5, batch_size=128, verbose=1, class_weight = class_weights, validation_split=0.2)

#model.fit([padded_questions, padded_answers], labels, batch_size=32,
 #         epochs=20, validation_split=0.1, class_weight = class_weights)
  
final_model.fit([padded_questions, padded_answers], labels, epochs=8, batch_size=256, verbose=1, validation_split=0.1, class_weight = class_weights)

Train on 7804 samples, validate on 868 samples
Epoch 1/8
7804/7804 [==============================] - 133s 17ms/step - loss: 0.3904 - acc: 0.8812 - val_loss: 0.3887 - val_acc: 0.8698
Epoch 2/8
7804/7804 [==============================] - 130s 17ms/step - loss: 0.3660 - acc: 0.8812 - val_loss: 0.3876 - val_acc: 0.8698
Epoch 3/8
7804/7804 [==============================] - 129s 17ms/step - loss: 0.3681 - acc: 0.8812 - val_loss: 0.3890 - val_acc: 0.8698
Epoch 4/8
7804/7804 [==============================] - 130s 17ms/step - loss: 0.3670 - acc: 0.8812 - val_loss: 0.3867 - val_acc: 0.8698
Epoch 5/8
2048/7804 [======>.......................] - ETA: 1:31 - loss: 0.3400 - acc: 0.8936

KeyboardInterrupt: ignored

## Testing the trained model

#### Preparing the test data

In [0]:
test_questions = read_data(bigpath+"/test/a.toks", "questions")
test_answers = read_data(bigpath+"/test/b.toks", "answers")
test_labels = read_data(bigpath+"/test/sim.txt", "labels")
#question_ids = read_data(bigpath+"/dev/id.txt", "labels")

In [27]:
#test_seqs, max_test_seq_length = SEQ_make_qa_sequences_list(test_questions, test_answers)
padded_test_questions = SEQ_token_and_pad(test_questions, max_hidden_length)
padded_test_answers = SEQ_token_and_pad(test_answers, max_hidden_length)

[[132, 107992, 69404, 33, 29575, 0, 9, 154], [132, 107992, 69404, 33, 29575, 0, 9, 154], [132, 107992, 69404, 33, 29575, 0, 9, 154], [132, 107992, 69404, 33, 29575, 0, 9, 154], [132, 107992, 69404, 33, 29575, 0, 9, 154], [132, 107992, 69404, 33, 29575, 0, 9, 154], [132, 0, 369, 5288, 1001], [132, 0, 369, 5288, 1001], [132, 0, 369, 5288, 1001], [132, 0, 369, 5288, 1001], [132, 0, 369, 5288, 1001], [132, 0, 369, 5288, 1001], [132, 145, 8, 8315, 240217, 56, 69, 101, 228861], [132, 145, 8, 8315, 240217, 56, 69, 101, 228861], [132, 145, 8, 8315, 240217, 56, 69, 101, 228861], [132, 145, 8, 8315, 240217, 56, 69, 101, 228861], [132, 145, 8, 8315, 240217, 56, 69, 101, 228861], [132, 17, 18858, 219, 0], [132, 17, 18858, 219, 0], [132, 17, 18858, 219, 0], [132, 17, 18858, 219, 0], [132, 17, 18858, 219, 0], [132, 17, 18858, 219, 0], [132, 17, 18858, 219, 0], [132, 17, 18858, 219, 0], [132, 17, 18858, 219, 0], [132, 17, 18858, 219, 0], [132, 17, 18858, 219, 0], [132, 17, 18858, 219, 0], [132, 17, 1

In [0]:
padded_seqs.shape


(1130, 324)

#### Predicting some q, a pairs

In [0]:
test_preds = final_model.predict([padded_test_questions, padded_test_answers])

In [29]:
test_preds

array([[0.15468398],
       [0.15468398],
       [0.15468398],
       ...,
       [0.15468398],
       [0.15468398],
       [0.15468398]], dtype=float32)

In [30]:
i=15 #test example to check

# print(padded_test_seqs[1])
# print(padded_test_seqs[16])

print(test_preds[i])
print(test_questions[i])
print(test_answers[i])
print("correct label: "+str(test_labels[i]))

[0.15468395]
how old was sue lyon when she made lolita

the actress who played lolita , sue lyon , was fourteen at the time of filming .

correct label: 1
